# Optimizing Outbound Channel
## Example
In this example, we simulate a situation where we have already built and deployed 3 models that we use to score each customer's
- probability to respond to a TEXT campaign
- probability to respond to an EMAIL campaign
- probability to respond to an APP notification

We assume that for a new campaign, we want to choose the best marketing campaign and maximize response.

In [1]:
import pandas as pd
from pathlib import Path
import os

# set path directories
curr_dir = Path(os.getcwd())
#print('Current Directory is: ', str(curr_dir))
#data_dir = Path(curr_dir.parents[0] / 'data/')
data_dir = Path(curr_dir)

In [2]:
filename = 'Optimizing Outbound Channel.csv'

-------------------
Let's take a peek at our dataset:

In [3]:
df100k = pd.read_csv(Path(data_dir) / filename, index_col='customerid')
df100k.head(5)

,prob_respond_app,prob_respond_text,prob_respond_email
customerid,,,
1,0.072626,0.024626,0.020666
2,0.075903,0.060459,0.011163
3,0.068856,0.052904,0.025183
4,0.048639,0.000100,0.016238
5,0.080576,0.035682,0.014966


What we need to do, is simply choose which choice will yield the maximum probability of response? 

**Option A:** At face value, this is a simple problem: We would simply choose the highest maximum probability, and go with that.

What often happens in the real world, however, is that one particular channel might have a dramatically lower overall repsonse rate than the others. So simply using the maximum likelihood would render an entire marketing channel useless. 

A reasonable response from the business might be, _I know that it's not perfect, but I'm going to send at least 5,000 emails in the next campaign._

Another realistic scenario at larger organizations, is that the budgets for emailing and texting has already been decided. We must help all of the 3 channels at the same time, without having them fight over the same customers (a person in the Top percentile of email response is also probably very high in text response as well).

**Option B** is to normalize the predictive scores around the same scale before doing the comparison. This does yield a better balance for the final result, but it only does a _relative_ comparison. A customer who is in the 1st decile of email will be selected to receive it, on the basis that they are only in the 20th decile for text. The fact may remain, however, that this customer is still more likely to respond to text and we are sub-optimizing based on a relative ranking and not on true probability.

**Option C** is the one we will demonstrate, which is to leave the probabilities raw, and use optimization libaries to search for a solution which maximizes overall response, while being able to achieve various constraints that the business often tries to apply. Such as,
- No matter what, I only want to target 40,000 of my total customers
- I want to send at least 5,000 emails
- I can send unlimited APP notifications because they're free
- I can afford up to 25,000 text messages
----------------------------

For each customer, I only have 4 choices of what to do:


In [4]:
choices = [
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1]
]

In this notation, we're considering an array in the same order. So, 
- `[0,0,0]` represents sending nothing
- `[1,0,0]` represents sending an ad via APP notification
- `[0,1,0]` represents sending an ad via TEXT
- `[0,0,1]` represents sending an ad via EMAIL



In [5]:
# the following rules are going to be used by several options:
def set_rules(SIZE_TO_TEST):
    
    rules = {'maximum': 
             {'prob_respond_app': SIZE_TO_TEST,
              'prob_respond_text': int(SIZE_TO_TEST * 0.10),
              'prob_respond_email': SIZE_TO_TEST
             },
             'minimum': 
             {'prob_respond_app': int(SIZE_TO_TEST * 0.05),
             'prob_respond_text': int(SIZE_TO_TEST * 0.05),
             'prob_respond_email': int(SIZE_TO_TEST * 0.05)
             },
             'equals': {'overall_count': int(SIZE_TO_TEST * 0.40)}
            }
    
    #TODO:  Add error checking
    
    #- so that minimums added up cannot exceed size
    
    return rules

----------------------

## Method 1: LP (simplex)

We can use `scipy.optimize.linprog` to solve this linear optimization problem. It requires to setup the boundary conditions as matrix products, as outlined in the docs.

For an intuitive explanation of how this problem is setup for LP, review (link)[this notebook].

In [6]:
import time
import numpy as np
from scipy.optimize import linprog

SIZE_TO_TEST = 1000

df = df100k[:SIZE_TO_TEST] # limit df for testing
nc = df.shape[1]  # number of choices
rules = set_rules(SIZE_TO_TEST)

data = df.to_numpy().ravel()

############# CONTRAINTS ##############
# Max choices per customer is 1
A_ub_1 = np.zeros((len(df), len(data)))
for i in range(len(A_ub_1)):
    A_ub_1[i, nc*i:nc*(i+1)] = 1
b_ub_1 = np.ones(len(df))

#TODO: Later, make a dictionary here of min/max and 0/1/2 and limit

# Max. choices for APP
A_ub_2 = np.zeros((1, len(data)))
A_ub_2[0, 0::nc] = 1
b_ub_2 = np.array([rules['maximum']['prob_respond_app']])

# Max. choices for TEXT
A_ub_3 = np.zeros((1, len(data)))
A_ub_3[0, 1::nc] = 1
b_ub_3 = np.array([rules['maximum']['prob_respond_text']])

# Max. choices for EMAIL
A_ub_4 = np.zeros((1, len(data)))
A_ub_4[0, 2::nc] = 1
b_ub_4 = np.array([rules['maximum']['prob_respond_email']])

# Min. choices for APP
A_ub_5 = np.zeros((1, len(data)))
A_ub_5[0, 2::nc] = -1 # invert, minimum≠
b_ub_5 = np.array([-rules['minimum']['prob_respond_app']]) #-1,minimum

# Min. choices for TEXT
A_ub_6 = np.zeros((1, len(data)))
A_ub_6[0, 2::nc] = -1 # invert, minimum≠
b_ub_6 = np.array([-rules['minimum']['prob_respond_text']]) #-1,minimum

# Min. choices for EMAIL
A_ub_7 = np.zeros((1, len(data)))
A_ub_7[0, 2::nc] = -1 # invert, minimum≠
b_ub_7 = np.array([-rules['minimum']['prob_respond_email']]) #-1,minimum

# Total sum of choices
A_eq = np.ones((1, len(data)))
b_eq = np.array([rules['equals']['overall_count']])

#############   EXECUTE   ##############
start_time = time.time()
result = linprog(
    -1 * data,  # linprog aims to minimize the value
    A_eq=A_eq, b_eq=b_eq,
    A_ub=np.concatenate((A_ub_1, A_ub_2, A_ub_3, A_ub_4, A_ub_5, A_ub_6, A_ub_7), axis=0),
    b_ub=np.concatenate((b_ub_1, b_ub_2, b_ub_3, b_ub_4, b_ub_5, b_ub_6, b_ub_7), axis=0),
    bounds=(0, 1)
)
print(result.message, '\n')
print("--- " + str(SIZE_TO_TEST) + " rows in %s seconds ---" % round(time.time() - start_time))

#############    RESULTS    ##############
selected = pd.DataFrame((result.x.reshape(-1, 3) > 1e-6).astype(int), columns=['choose_app','choose_text','choose_email'])
df = df.reset_index(drop=False)
method1 = pd.concat([df,selected], axis=1, ignore_index=False)

method1[['choose_app','choose_text','choose_email']].agg(['sum'])

Optimization terminated successfully. 

--- 1000 rows in 102 seconds ---


,choose_app,choose_text,choose_email
sum,250,100,50


In [7]:
score = ((method1['prob_respond_app'] * method1['choose_app'] + method1['prob_respond_text'] * method1['choose_text'] + method1['prob_respond_email'] * method1['choose_email'])).sum()
print("The overall maximization score is: " + str(score))

The overall maximization score is: 31.912364306


In [8]:
method1[(method1['choose_app'] > 0) | (method1['choose_text'] > 0) | (method1['choose_email'] > 0)].head(8)

,customerid,prob_respond_app,prob_respond_text,prob_respond_email,choose_app,choose_text,choose_email
0,1,0.072626,0.024626,0.020666,1,0,0
1,2,0.075903,0.060459,0.011163,1,0,0
2,3,0.068856,0.052904,0.025183,1,0,0
4,5,0.080576,0.035682,0.014966,1,0,0
8,9,0.083755,0.018927,0.007464,1,0,0
11,12,0.075007,0.022823,0.021770,1,0,0
12,13,0.085423,0.050261,0.014796,1,0,0
18,19,0.046501,0.102492,0.028378,0,1,0


By iterating between 100 and 2000 observations (always keeping the constraints to similar ratios of the data), I observed that the processing time increases with a power growth, which is a concern to be able to use this in the real world.

Initial tests show that scoring on 100,000 customers would take about 370 days on my laptop.

Next, we'll try another method to compare the performance.

### Method 2: Multivariate Optimization

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import basinhopping
from itertools import chain

In [10]:
SIZE_TO_TEST = 1000
solver = 'SLSQP'

########################################
noCustomers = SIZE_TO_TEST
rules = set_rules(SIZE_TO_TEST)
df = df100k[:noCustomers].reset_index(drop=False)

data = np.array(df.values.tolist())


########  OBJECTIVE FUNCTION ##########

# MAXIMIZE THE SUM OF PROBABILITIES ACROSS ALL CUSTOMERS AND TREATMENTS
def objective(x):
    return -(data[:, 1:] * x.reshape(3,-1).T).sum()


###########  CONSTRAINTS #############

cons = []

#TOTAL CONTACTS FROM APP/TEXT/EMAIL ARE LESS THAN 40,000
def constraintTot(x):
    return rules['equals']['overall_count'] - x.sum()
conTot = {'type':"ineq","fun":constraintTot}
cons.append(conTot) 

# Max. choices for APP
def constraintAppMax(x):
    return rules['maximum']['prob_respond_app'] - (data[:, 1:2] * x.reshape(3,-1).T[:,0:1]).sum()
conAppMax = {'type':"ineq","fun":constraintAppMax}
cons.append(conAppMax) 

# Max. choices for TEXT
def constraintTextMax(x):
    return rules['maximum']['prob_respond_text'] - (data[:, 2:3] * x.reshape(3,-1).T[:,1:2]).sum()
conTextMax = {'type':"ineq","fun":constraintTextMax}
cons.append(conTextMax) 

# Max. choices for EMAIL
def constraintEmailMax(x):
    return rules['maximum']['prob_respond_email'] - (data[:, 3:4] * x.reshape(3,-1).T[:,2:3]).sum()
conEmailMax = {'type':"ineq","fun":constraintEmailMax}
cons.append(conEmailMax) 

# Min. choices for APP
def constraintAppMin(x):
    return -rules['minimum']['prob_respond_app'] + (data[:, 1:2] * x.reshape(3,-1).T[:,0:1]).sum()
conAppMin = {'type':"ineq","fun":constraintAppMin}
cons.append(conAppMin) 

# Min. choices for TEXT
def constraintTextMin(x):
    return -rules['minimum']['prob_respond_text'] + (data[:, 2:3] * x.reshape(3,-1).T[:,1:2]).sum()
conTextMin = {'type':"ineq","fun":constraintTextMin}
cons.append(conTextMin) 

# Min. choices for EMAIL
def constraintEmailMin(x):
    return -rules['minimum']['prob_respond_email'] + (data[:, 3:4] * x.reshape(3,-1).T[:,2:3]).sum()
conEmailMin = {'type':"ineq","fun":constraintEmailMin}
cons.append(conEmailMin) 

#FORCE EACH CUSTOMER TO ONLY HAVE ONE TREATMENT OUT OF TEXT/EMAIL/APP
def con2(a):
    def g2(x):
        return 1 - x[a] - x[a+noCustomers] - x[a+(2*noCustomers)]
    return g2
for t in range (0,noCustomers):
    cons.append({'type':'ineq', 'fun': con2(t)})
    
#INITIAL SOLUTION - SET EVERYTHING TO ZERO
x0 = np.array([[0] * noCustomers + [0] * noCustomers + [0] * noCustomers ])

#BOUNDS FOR ALL DECISION VARIABLES
b= (0,1)
bnds = ()
for i in range(noCustomers*3):
    bnds = bnds + (b,)

############# optimization ###################
start_time = time.time()
sol = minimize(objective,x0,method=solver,constraints=cons,bounds=bnds)
print(result.message, '\n')
print("--- " + str(SIZE_TO_TEST) + " rows in %s seconds ---" % round(time.time() - start_time))

################# results ################
customers = list(chain.from_iterable(np.array(data)[:,0:1].tolist()))
app = sol.x[0:noCustomers].tolist()
text = sol.x[noCustomers:2*noCustomers].tolist()
email = sol.x[2*noCustomers:3*noCustomers].tolist()

df1 = pd.DataFrame(np.array(customers + app + text + email).reshape(4,noCustomers).transpose(),\
                       columns=["customerid","choose_app","choose_text","choose_email"])
df1["choose_app"] = np.round(df1["choose_app"],decimals=0)
df1["choose_text"] = np.round(df1["choose_text"],decimals=0)
df1["choose_email"] = np.round(df1["choose_email"],decimals=0)
method2 = df.merge(df1,on="customerid",how="left")
method2[['choose_app','choose_text','choose_email']].agg(['sum'])

Optimization terminated successfully. 

--- 1000 rows in 24525 seconds ---


,choose_app,choose_text,choose_email
sum,85.0,77.0,238.0


In [11]:
score = ((method2['prob_respond_app'] * method2['choose_app'] + method2['prob_respond_text'] * method2['choose_text'] + method2['prob_respond_email'] * method2['choose_email'])).sum()
print("The overall maximization score is: " + str(score))

The overall maximization score is: 23.401941783


In [12]:
method2[(method2['choose_app'] > 0) | (method2['choose_text'] > 0) | (method2['choose_email'] > 0)].head(8)

,customerid,prob_respond_app,prob_respond_text,prob_respond_email,choose_app,choose_text,choose_email
8,9,0.083755,0.018927,0.007464,1.0,0.0,0.0
10,11,0.031328,0.057764,0.030572,0.0,0.0,1.0
12,13,0.085423,0.050261,0.014796,1.0,0.0,0.0
16,17,0.063193,0.069687,0.029876,0.0,0.0,1.0
17,18,0.052648,0.027838,0.027922,0.0,0.0,1.0
18,19,0.046501,0.102492,0.028378,0.0,1.0,0.0
22,23,0.057932,0.000100,0.034093,0.0,0.0,1.0
23,24,0.051683,0.079682,0.029333,0.0,0.0,1.0


### Method 3: Ordered-Sorting Method
This method essentially cascades through various constraints in a specific order, utilizing sorts to accumulate choices

- Step 1: Satisfy minimums first by choosing top N for each choice (relative ranking)
- Step 2: Choose from global optimum choices until first maximum constraint is met
- Step 3: Remove that choice, and iteratively repeat Step 2 until either total max is reached


In [13]:
import time
import numpy as np

# settings
SIZE_TO_TEST = 1000
WORST_TO_BEST_FLAG = True # True: Worst overall gets first choice  # False: Best overall gets priority
MAX_VARNAME = 'max_prob'
CHOSEN_SUFFIX = '_is_chosen'


# Given a dataframe of probs, choose the best option
def choose_best(DF):
    cols = [c for c in list(DF.columns) if MAX_VARNAME not in c and CHOSEN_SUFFIX not in c]
    newcols = []
    result = DF.copy()
    for c in cols:
        best_col = c + CHOSEN_SUFFIX
        result[best_col] = np.where((result[c]==result[MAX_VARNAME]), 1, 0)
        newcols.append(best_col)
    return result
        
# Given a dataframe of choices, create a running total of each choice
def order_choices(DF):
    cols = [c for c in list(DF.columns) if CHOSEN_SUFFIX in c]
    result = DF[cols].cumsum()
    result['total'] = result.sum(axis=1)
    return result

# Given a dataframe of probabilities, display as a relative ranking within each column
def show_as_rank(DF):
    original_cols_plus_max = list(DF.columns)
    if MAX_VARNAME not in original_cols_plus_max:
        original_cols_plus_max.append(MAX_VARNAME)
    
    result = pd.DataFrame(columns=original_cols_plus_max)

    for c in original_cols_plus_max:
        result[c] = DF[c].rank(ascending=False)
    return result

    


# initialize some parameters 
rules = set_rules(SIZE_TO_TEST) # rules
print(rules)
df = df100k[:SIZE_TO_TEST].copy() # limit df for testing
ORIGINAL_COLS = list(df.columns)
nc = df.shape[1]  # number of choices
df_avg_all = df[ORIGINAL_COLS].agg(['mean']).sort_values(by=['mean'], ascending=WORST_TO_BEST_FLAG, axis=1)
ordered_cols = list(df_avg_all.columns) # create list of columns in order of best to worst

start_time = time.time()

# add a column: maximum probability per customer
df[MAX_VARNAME] = df.max(axis=1)
df.sort_values(by=[MAX_VARNAME], axis=0, ascending=False)

# choose the most optimum scenario which is the best overall
best = choose_best(df)
    
# create a dataset represented as relative rank within each choice
df_rank = show_as_rank(df)

# need to keep track of items we select throughout different iterations and logic
keep_details = {} # will refer to this later on which index receives which option
keep_indexes = pd.Int64Index([], dtype='int64') # saving indexes separately

# SATISFY MINIMUMS FIRST: in order of best overall to worst, select the best of each to satisfy minimums
# NOTE: important to go in order (this is where this process becomes sub-optimum)
for c in ordered_cols:
    temp = df_rank.nsmallest(rules['minimum'][c],c).index
    df_rank.drop(temp,inplace=True)
    keep_details[c] = temp

for i in keep_details.values():
    keep_indexes = keep_indexes.union(i)

#print('Found ' + str(len(keep_indexes)) + ' customers to satisfy minimum constraints.')

# update maximums in the ruleset
rules = set_rules(SIZE_TO_TEST)
for k, v in rules['maximum'].items():
    rules['maximum'][k] -= len(keep_details[k])
    
rules['equals']['overall_count'] -= len(keep_indexes)


# CHOOSE OPTIMAL CHOICE UNTIL MAX IS SATISFIED
while rules['equals']['overall_count'] != 0:
    
    # whatever is left, we store by separately
    leftovers = df.drop(keep_indexes).copy().sort_values(by=[MAX_VARNAME], ascending=False, axis=0)

    best = choose_best(leftovers)

    choices = order_choices(best)

    zero_key1 = [k for k, v in rules['maximum'].items() if v == 0]
    zero_key2 = [k + CHOSEN_SUFFIX for k, v in rules['maximum'].items() if v == 0]

    if not len(zero_key2)==0:
        leftovers.drop(columns=zero_key1, axis=1, inplace=True)
        leftovers.drop(columns=MAX_VARNAME, axis=1, inplace=True)

        # refresh the max probability per customer
        leftovers[MAX_VARNAME] = leftovers.max(axis=1)
        leftovers.sort_values(by=[MAX_VARNAME], axis=0, ascending=False)

        best = choose_best(leftovers)
        choices = order_choices(best)
        for c in zero_key2:
            choices[c]=0


    chosen = choices[(choices['prob_respond_app' + CHOSEN_SUFFIX] <= rules['maximum']['prob_respond_app']) & \
            (choices['prob_respond_text' + CHOSEN_SUFFIX] <= rules['maximum']['prob_respond_text']) & \
            (choices['prob_respond_email' + CHOSEN_SUFFIX] <= rules['maximum']['prob_respond_email']) & \
            (choices['total'] <= rules['equals']['overall_count'])]

    #print('... added ' + str(len(chosen)))

    # log which indexes we chose
    for c in rules['maximum']:
        #print(chosen.columns)
        if c in list(best.columns):
            #! This line drives the Boolean Series Key warning! ?!??!
            subset = best[best.index.isin(chosen[best[c + CHOSEN_SUFFIX]==1].index)]
            i = keep_details[c]
            i = i.append(pd.Int64Index(subset.index, dtype='int64'))
            keep_details[c]=i


    i = keep_indexes
    i = i.append(pd.Int64Index(chosen.index, dtype='int64'))
    keep_indexes = i

    # update maximums in the ruleset
    rules = set_rules(SIZE_TO_TEST)
    for k, v in rules['maximum'].items():
        rules['maximum'][k] -= len(keep_details[k])

    rules['equals']['overall_count'] -= len(keep_indexes)

    #print('...... ' + str(len(keep_indexes)) + ' so far')

print("--- " + str(SIZE_TO_TEST) + " rows in %s seconds ---" % round(time.time() - start_time))


df = df.sort_values(by=['max_prob'], ascending=False, axis=0)
df.drop(columns=['max_prob'], axis=1, inplace=True)

result = df.copy()

for k, v in keep_details.items():
    result[k] = np.where((result.index.isin(v)), 1, 0)

result.columns=['choose_app','choose_text','choose_email']    
method3 = df.merge(result,on="customerid",how="left")
method3[['choose_app','choose_text','choose_email']].agg(['sum'])

{'maximum': {'prob_respond_app': 1000, 'prob_respond_text': 100, 'prob_respond_email': 1000}, 'minimum': {'prob_respond_app': 50, 'prob_respond_text': 50, 'prob_respond_email': 50}, 'equals': {'overall_count': 400}}
--- 1000 rows in 0 seconds ---


/Users/josh.berry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/josh.berry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/josh.berry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/josh.berry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/josh.berry/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,choose_app,choose_text,choose_email
sum,247,100,53


In [14]:
score = ((method3['prob_respond_app'] * method3['choose_app'] + method3['prob_respond_text'] * method3['choose_text'] + method3['prob_respond_email'] * method3['choose_email'])).sum()
print("The overall maximization score is: " + str(score))

The overall maximization score is: 30.64108377


In [15]:
method3[(method3['choose_app'] > 0) | (method3['choose_text'] > 0) | (method3['choose_email'] > 0)].head(8)

,prob_respond_app,prob_respond_text,prob_respond_email,choose_app,choose_text,choose_email
customerid,,,,,,
559,0.061931,0.150913,0.039815,0,0,1
644,0.065695,0.149535,0.024563,0,1,0
125,0.042633,0.138856,0.018009,0,1,0
910,0.058696,0.128025,0.013692,0,1,0
866,0.019872,0.127482,0.023886,0,1,0
504,0.073833,0.123875,0.020419,0,1,0
524,0.056580,0.123608,0.024210,0,1,0
414,0.009328,0.121295,0.026430,0,1,0


# Method 4: Ant Colony Optimizer (Work in Progress)

In [ ]:
class AntColony:
    """A Ant Colony Optimizer for constrained feature optimization.
    https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms
    Parameters
    ----------
    data : array
        An array of choices to be selected. One per row.
    constraints : list of lists
        For each option, a lower and upper range of how often it may occur.
    alpha: float
        Hyperparameter controlling pheromones, higher alpha gives pheromones more weight.
    decay: float
        Hyperparameter controlling pheromones decay.
    n_ants: int
        Number of ants per iteration.
    n_best: int
        Number of best ants who update pheromones.
    max_iterations: int
        Maximum number of iterations.
    empty_rounds: int
        Maximum number of iteration allowed without finding a global new best. (Early stopping).
    constraints: dict
        Dict with constraint names as keys and possible choices as values.
        All constraints must be categorical.
    fixed: dict
        Fixed parameters.
    minimize: bool
        Whether to minimize of maximize.
    """

    def __init__(
        self,
        data,
        constraints,
        alpha=0.5,
        decay=0.9,
        n_ants=100,
        n_best=10,
        max_iterations=50,
        empty_rounds=10,
        minimize=False,
    ):
        self.alpha = alpha
        self.decay = decay
        self.n_ants = n_ants
        self.n_best = n_best
        self.max_iterations = max_iterations
        self.empty_rounds = empty_rounds
        self.minimize = minimize
        
        self.constraints = constraints
        self.data = data

        self.n_rows, self.n_columns = self.data.shape
        self.idxs = np.arange(self.n_rows).astype(int)
        # array where each ant's choice for 1 iteration will be stored
        self.choices_per_ant = np.empty((self.n_ants, self.n_rows), dtype=int)
        # dict with constraints as keys and number of categories/choices as values
        self.pheromones = np.ones(self.data.shape)
        self.simulations = []
        self.best_total_idx = 0
        self.best_total_value = np.inf if minimize else -np.inf
        self.no_improvement = 0
        
        self.int_options = np.arange(self.n_columns).astype(int)

        print(
            f'Initializing the ant colony with {self.n_ants} workers '
            f'and {self.n_best} best ants.'
        )

    def pick_move(self):
        """
        Select categories for each ant and construct a DataFrame with selected results
        Return
        ------
        DataFrame with selected categories for each ant
        """
        for idx in range(self.n_rows):
            p = self.pheromones[idx] ** self.alpha
            self.choices_per_ant[:, idx] = np.random.choice(
                self.int_options, self.n_ants, p=p / p.sum()
            )

    def update_global_bests(self, best_iter_value, best_iter_idx):
        """
        Update global best value and keeps track of improvements for early stopping
        """
        improved = (
            best_iter_value < self.best_total_value
            if self.minimize
            else best_iter_value > self.best_total_value
        )
        if improved:
            self.best_total_value = best_iter_value
#            self.best_total_idx = len(self.simulations) + best_iter_idx
            self.best_choice = self.choices_per_ant[best_iter_idx]
            self.no_improvement = 0
            print(f'New global best found: {self.best_total_value}')
        else:
            self.no_improvement += 1

    def update_best_ants(self, results, epsilon=0.01):
        """
        Extract indexes of best ants as well as best iteration value
        Return
        ------
        Tuple with normalized_results, list of best ants indices and best iteration value
        """
        sorted_results_idx = results.argsort()
        # normalizing to [0 + epsilon, 1 + epsilon] to avoid division by zero in `update_pheromones`
        normalized_results = epsilon + results / results.max()
        if results.max() - results.min() != 0:
            normalized_results = epsilon + (results - results.min()) / (
                results.max() - results.min()
            )
        if self.minimize:
            # the first item in sorted results will be the smallest
            best_ants = sorted_results_idx[: self.n_best]
            best_iter_idx = 0
            best_iter_value = results[best_ants][best_iter_idx]
            normalized_results[sorted_results_idx[self.n_best :]] = np.inf  # noqa: E203
        else:
            # the last item in sorted results will be the largest
            best_ants = sorted_results_idx[-self.n_best :]  # noqa: E203
            best_iter_idx = self.n_best - 1
            best_iter_value = results[best_ants][best_iter_idx]
            normalized_results[sorted_results_idx[: -self.n_best]] = 0
        self.update_global_bests(best_iter_value, best_iter_idx)
        return normalized_results, best_ants, best_iter_value

    def update_pheromones(self, results):
        """
        Update pheromones based on results from best ants
        """
        for idx in range(self.n_rows):
            p = self.pheromones[idx]
            p *= self.decay
            choices = self.choices_per_ant[:, idx]
            p[choices] += (1 / results) if self.minimize else results

    def append_simulations(self, df, best_ants_idx, results):
        """
        Append simulation. Each best ant's iteration is stored
        """
        for i in best_ants_idx:
            row = df.loc[i].to_dict()
            simulation = {
                'features': [
                    {'name': name, 'value': convert_to_native(value)} for name, value in row.items()
                ],
                'prediction': convert_to_native(results[i]),
            }
            self.simulations.append(simulation)

    def one_iter(self):
        """
        One iteration of the algorithm:
        1. Select values for each ant.
        2. Do a batch prediction.
        3. Select top `self.n_best` ants.
        4. Update pheromones.
        5. Append best simulations.
        """
        self.pick_move()
        results = self.evaluate()
        normalized_results, best_ants_idx, best_iter_value = self.update_best_ants(results)
        self.update_pheromones(normalized_results)
        #self.append_simulations(df, best_ants_idx, results)
        return best_iter_value

    def evaluate(self):
        results_per_ant = np.zeros(self.n_ants)
        for idx, this_ant in enumerate(self.choices_per_ant):
            values = self.data[self.idxs, this_ant]
            results_per_ant[idx] = values.sum()
        for cat, (lower, upper) in enumerate(self.constraints):
            counts = (self.choices_per_ant == cat).sum(1)
            results_per_ant[(lower > counts) + (upper < counts)] /= 10
        return results_per_ant
            #self.objective

    def optimize(self):
        """
        Main loop of ACO
        """
        for iteration in range(self.max_iterations):
            result = self.one_iter()
#            print(f'Iteration {iteration}. Best score: {result}')
            if self.no_improvement > self.empty_rounds:
                print(f'No improvement in {self.empty_rounds} iterations. Stopping early')
                break

        return
        optimized_simulation = self.simulations[self.best_total_idx].copy()
        optimized_simulation['index'] = convert_to_native(self.best_total_idx)
        return {
            'prediction': optimized_simulation['prediction'],
            'simulations': self.simulations,
            'optimized_simulation': optimized_simulation,
        }

In [ ]:
constraints = [[10, 90], [10, 90], [10, 90]]  # each category may occur between 10 and 90 times
vals = df.iloc[:, 1:].to_numpy()

In [ ]:
self = AntColony(vals, constraints, empty_rounds=500, max_iterations=1000, n_ants=1000)

In [ ]:
self.optimize()

In [ ]:
self.best_choice

In [ ]:
from collections import Counter
Counter(self.best_choice)

In [ ]:
df.sum()

In [ ]:
vals.max(1).sum()

In [ ]:
####### JUNK DRAWER:  IGNORE EVERYTHING BELOW THIS LINE #########

#### Testing Performance

In [ ]:
import time
import numpy as np
from scipy.optimize import linprog

testresults = []

for s in range(100,2000,100):

    SIZE_TO_TEST = s

    # which dataframe to run on (for testing of performance)
    df = df100k[:SIZE_TO_TEST]

    # number of choices
    nc = df.shape[1]  

    MAX_APP = SIZE_TO_TEST #df.shape[0]
    MAX_TEXT = int(SIZE_TO_TEST * 0.25)
    MIN_EMAIL = int(SIZE_TO_TEST * 0.05)
    TOTAL_N = int(SIZE_TO_TEST * 0.40)

    data = df.to_numpy().ravel()

    ############# CONTRAINTS ##############
    # Max choices per customer is 1
    A_ub_1 = np.zeros((len(df), len(data)))
    for i in range(len(A_ub_1)):
        A_ub_1[i, nc*i:nc*(i+1)] = 1
    b_ub_1 = np.ones(len(df))

    #TODO: Later, make a dictionary here of min/max and 0/1/2 and limit

    # Max. choices for APP
    A_ub_2 = np.zeros((1, len(data)))
    A_ub_2[0, 0::nc] = 1
    b_ub_2 = np.array([MAX_APP])

    # Max. choices for TEXT
    A_ub_3 = np.zeros((1, len(data)))
    A_ub_3[0, 1::nc] = 1
    b_ub_3 = np.array([MAX_TEXT])

    # Min. choices for EMAIL
    A_ub_4 = np.zeros((1, len(data)))
    A_ub_4[0, 2::nc] = -1 # invert, minimum
    b_ub_4 = np.array([-MIN_EMAIL]) #-1,minimum

    # Total sum of choices
    A_eq = np.ones((1, len(data)))
    b_eq = np.array([TOTAL_N])

    start_time = time.time()
    result = linprog(
        -1 * data,  # linprog aims to minimize the value
        A_eq=A_eq, b_eq=b_eq,
        A_ub=np.concatenate((A_ub_1, A_ub_2, A_ub_3, A_ub_4), axis=0),
        b_ub=np.concatenate((b_ub_1, b_ub_2, b_ub_3, b_ub_4), axis=0),
        bounds=(0, 1)
    )
    print(result.message, '\n')
    
    test = [str(s), str(time.time() - start_time)]
    testresults.append(test)
    print("--- %s seconds ---" % (time.time() - start_time))

    # optimum selections
    selected = pd.DataFrame((result.x.reshape(-1, 3) > 1e-6).astype(int), columns=['choose_app','choose_text','choose_email'])

    df = df.reset_index(drop=False)

    final = pd.concat([df,selected], axis=1, ignore_index=False)


    #final[['choose_app','choose_text','choose_email']].agg(['sum'])

print(test)
    

In [ ]:
testresults

In [ ]:
number_of_rows = 100000
time_to_run = 0.000000047733 * number_of_rows**2.965362611291
time_to_run